In [1]:
import strax
import pandas as pd
pd.options.display.max_colwidth = 100
from tqdm import tqdm
import numpy as np
import numba
from scipy.optimize import curve_fit
from scipy import stats
import straxen
st = straxen.contexts.strax_workshop_dali()

#st.data_info('raw_records')

In [2]:
runs = st.select_runs(run_mode='LED*')
#print(runs.tags, runs.name)

Checking data availability: 100%|██████████| 5/5 [00:11<00:00,  2.45s/it]


In [3]:
noise_id = '180219_0952'
run_id = '180219_1011'

noise = st.get_array(noise_id, 'raw_records', seconds_range=(0, 20))
df = st.get_array(run_id, 'raw_records', seconds_range=(0, 20))

In [4]:
def gaus(x, a, mu, sig, const):
    return a*np.exp(-0.5*(x-mu)**2/sig**2)+const

In [5]:
def get_LED_window(raw_records):
    
    info = {'mean': [ ], 'sig': [ ], 'Norm': [ ], 'Const': [ ], 'pmt': [ ]}
    d2 = pd.DataFrame({'channel': [ ], 'idx_LED': [ ]}) 
    n_channel_s = np.arange(0, 249, 1)
    pmts_rejected = [ ]
    x = np.arange(0,len(raw_records['data']),1)

    #for n_channel in tqdm(n_channel_s):
    for n_channel in n_channel_s:
                #if n_channel in pmts_rejected:
                 #     continue   
                wf_tmp = raw_records[raw_records['channel'] == n_channel]
                amp_wf = [ ]
                idx_amp_wf = [ ]
                for wf_ in wf_tmp:
                    amp_wf.append(np.max(wf_['data']))
                    idx_amp_wf.append(np.argmax(wf_['data']))
                    
                amp_wf = np.array(amp_wf)
                idx_amp_wf = np.array(idx_amp_wf)

                hist, xbins = np.histogram(amp_wf, bins=100, range=(0,200))
                xbins_center = np.array([0.5*(xbins[i]+xbins[i+1]) for i in range(len(xbins)-1)])

                mask = (xbins_center > 30) & (xbins_center < 200)
                popt, pcov = curve_fit(gaus, xbins_center[mask], hist[mask], p0=[10, 60, 20, 0], maxfev=int(1e6))

                N, mean, sig, c = popt[0], popt[1], popt[2], popt[3]

                info['mean'].append(popt[1])
                info['sig'].append(popt[2])
                info['Norm'].append(popt[0])
                info['Const'].append(popt[3])
                info['pmt'].append(n_channel)

                d1 = pd.DataFrame({'channel': [ ], 'idx_LED': [ ]})
                mask = (amp_wf < mean + sig) & (amp_wf > mean - sig)
                idx_LED = idx_amp_wf[mask]
                if len(idx_LED)==0:
                    d1['channel'] = n_channel
                    d1['idx_LED'] = np.nan
                else:
                    d1['idx_LED'] = idx_LED
                    d1['channel'] = np.ones_like(idx_LED) * n_channel
                    
                d2 = d2.append(d1, ignore_index=True)
                del d1, idx_amp_wf, idx_LED, amp_wf
                  
    mean = d2['idx_LED'].mean()
    std = d2['idx_LED'].std()
    window = [int(mean-0.5*std),int(mean+0.5*std)]
    return info, window

In [6]:
info, window = get_LED_window(df)

/dali/lgrandi/strax/miniconda3/envs/strax/lib/python3.6/site-packages/scipy/optimize/minpack.py:799: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [7]:
def get_amplitude(raw_records, window):  
    on = []
    off = []
    for r in raw_records:
        amp_LED = np.max(r['data'][window[0]:window[1]])
        amp_NOISE = np.max(r['data'][2*window[0]:2*window[1]])
        on.append(amp_LED)
        off.append(amp_NOISE)
    on = np.array(on, dtype=[('amplitudeLED', '<i4')])
    off = np.array(off, dtype=[('amplitudeNOISE', '<i4')])
    return on, off

def get_area(raw_records, window):

    left = window[0]
    end_pos = [window[1]+2*i for i in range(6)]
    n_channel_s = np.arange(0, 249, 1)
    Area = np.array(raw_records[['channel', 'area']],dtype=[('channel','int16'),('area','float32')])
                    
    for n_channel in n_channel_s:
        wf_tmp = raw_records[raw_records['channel'] == n_channel]
        area = 0
        for right in end_pos:
            area += wf_tmp['data'][:,left:right].sum(axis=1)
            
        mask = np.where(Area['channel'] == n_channel)[0]
        Area['area'][mask] = area.astype(np.float)/6.
        #print(area, Area['area'][mask])

    return Area

def get_tags(runs):
    df = pd.DataFrame(np.transpose([runs.tags.values, runs.name.values]), columns=['Tags', 'Name'])
    return df

@strax.takes_config(
    strax.Option(
        'LED_window',
        default=(125, 250),
        help="Window (samples) where we expect signal in LED calibration"))

class LEDCalibration(strax.Plugin):
    '''
    Let's start with something really easy.
    LEDCalibration gives channel, data, amp_LED and amp_NOISE
    '''   
    __version__ = '0.0.3'

    depends_on = ('raw_records',)
    data_kind = 'led_cal_0'
    compressor = 'zstd'
    parallel = 'process'
    rechunk_on_save = False
    dtype = [('area', np.int32, 'Area'),
             ('amplitudeLED', np.int32, 'Amplitude in LED window'),
             ('amplitudeNOISE', np.int32, 'Amplitude in off LED window'),
             ('channel', np.int16, 'Channel'),
             ('time', np.int64, 'Start time of the interval (ns since unix epoch)'),
             ('dt', np.int16, 'Time resolution in ns'),
             ('length', np.int32, 'Length of the interval in samples')
            ]

    def compute(self, raw_records):
        # Remove records from funny channels (if present)
        r = raw_records[raw_records['channel'] < 248] # hardcoded for now
        temp = np.zeros(len(r), dtype=self.dtype)
        
        temp['channel'] = r['channel']
        temp['time'] = r['time']
        temp['dt'] = r['dt']
        temp['length'] = r['length']
        
        #info, window = get_LED_window(r)
        #print(window)
        #on, off = get_amplitude(r, window) 
        on, off = get_amplitude(r, self.config['LED_window'])
        temp['amplitudeLED'] = on
        temp['amplitudeNOISE'] = off
        #area = get_area(r, window)
        area = get_area(r, self.config['LED_window'])
        #print(area['channel'] == r['channel'])
        temp['area'] = area['area']
        
        return temp

In [8]:
st_2 = straxen.contexts.strax_workshop_dali()
st_2.register(LEDCalibration)
st_2.data_info('led_calibration')

,Field name,Data type,Comment
0,area,int32,Area
1,amplitudeLED,int32,Amplitude in LED window
2,amplitudeNOISE,int32,Amplitude in off LED window
3,channel,int16,Channel
4,time,int64,Start time of the interval (ns since unix epoch)
5,dt,int16,Time resolution in ns
6,length,int32,Length of the interval in samples


In [ ]:
arr_tmp = st_2.get_array(run_id, 'led_calibration')

Removing old incomplete data in ./strax_data/180219_1011-led_calibration-gcdt23vryt


In [ ]:
arr_tmp

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

n_channel_s = np.arange(0, 10, 1)

for n_channel in tqdm(n_channel_s):
    arr = arr_tmp[arr_tmp['Channel'] == n_channel]
    #df_noise_tmp = df_noise[df_noise['Channel'] == n_channel]

    fig = plt.figure(figsize=(12,8))

    LED_amp_spectrum, bins_amp_1 = np.histogram(arr['amplitudeLED'], bins=150, range=(0,300))
    noise_amp_spectrum, bins_amp_2 = np.histogram(arr['amplitudeNOISE'], bins=150, range=(0,300))
    
    #LED_area_spectrum, bins_area_1 = np.histogram(df_LED_tmp['Area'], bins=150, range=(0,300))
    #noise_area_spectrum, bins_area_2 = np.histogram(df_noise_tmp['Area'], bins=150, range=(0,300))
    
    #plt.subplot(221)
    #plt.errorbar(x = bins_1[:len(bins_1)-1], y = LED_spectrum, yerr = np.sqrt(LED_spectrum), fmt='k+', label='LED')
    #plt.errorbar(x = bins_2[:len(bins_2)-1],  y = noise_spectrum_2, yerr = np.sqrt(noise_spectrum_2), fmt='r+', label='noise')
    #plt.vlines(x=ADC_correction, ymin=0, ymax=1e4, colors='k', linestyles='dashed')
    #plt.xlabel('amp (ADC counts)')
    #plt.title('Channel %d - ZOOM' %n_channel)
    #plt.yscale('log')
    #plt.legend(loc='best')
    #plt.xlim(0,50)
    #plt.plot([(bins_area_1[i]+bins_amp_1[i+1])*0.5 for i in range(len(bins_area_1)-1)], LED_area_spectrum, color='k', label='Area LED')
    #plt.plot([(bins_area_1[i]+bins_area_1[i+1])*0.5 for i in range(len(bins_area_1)-1)], noise_area_spectrum, color='r', label='Area noise')
    #plt.xlabel('Area (ADC counts x time)')
    #plt.title('Channel %d' %n_channel)
    #plt.yscale('log')
    #plt.legend(loc='best')
    #plt.xlim(0)
    
    plt.subplot(311)
    #plt.hist(df_LED_tmp['Amplitude'], bins=100, range=(0,200), label='LED')
    #plt.hist(df_noise_tmp['Amplitude'], bins=100, range=(0,200), label='noise')
    plt.plot([(bins_amp_1[i]+bins_amp_1[i+1])*0.5 for i in range(len(bins_amp_1)-1)], LED_amp_spectrum, color='k', label='Amp LED')
    plt.plot([(bins_amp_2[i]+bins_amp_2[i+1])*0.5 for i in range(len(bins_amp_2)-1)], noise_amp_spectrum, color='r', label='Amp noise')
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d' %n_channel)
    plt.yscale('log')
    plt.legend(loc='best')
    plt.xlim(0)
    
    
    ADC_correction = 7
    tmp_1 = arr[arr['amplitudeLED'] < ADC_correction]
    tmp_2 = arr[arr['amplitudeNOISE'] < ADC_correction]
    scaling_coeff = tmp_1['amplitudeLED'].sum()/tmp_2['amplitudeNOISE'].sum()
    noise_amp_spectrum_2 = noise_amp_spectrum*scaling_coeff

    plt.subplot(312)
    #hist_LED, xbins_LED = np.histogram(d1_LED['Amplitude'], bins=50, range=(0,50))
    #hist_noise, xbins_noise = np.histogram(d2_noise, bins=50, range=(0,50))
    x_center = [(bins_amp_1[i]+bins_amp_1[i+1])*0.5 for i in range(len(bins_amp_1)-1)]
    diff = abs(LED_amp_spectrum - noise_amp_spectrum_2)
    sigma_diff = np.sqrt(LED_amp_spectrum - noise_amp_spectrum_2)
    plt.errorbar(x = x_center, y=diff, yerr = sigma_diff, fmt='b+', label='Difference')
    #hist_diff, xbins_diff, _ = plt.hist(diff, bins=50, range=(0,50), label='Difference', histtype='step')
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d' %n_channel)
    plt.legend(loc='best')
    
    
    plt.subplot(313)
    res =  1. - np.cumsum(diff)/np.sum(diff)
    pos_15ADC = x_center.index(15)
    acc_15ADC = res[pos_15ADC]*100
    plt.plot(x_center, res, label='Acceptance @ 15 ADC = %.2f'%(acc_15ADC))
    plt.title('Acceptance')
    plt.ylim(0,1.1)
    plt.xlim(0, 100)
    plt.xlabel('amp (ADC counts)')
    plt.title('Channel %d' %n_channel)
    plt.legend(loc='best')
    
    plt.tight_layout()
    plt.show()
    
    #del df_LED_tmp, df_noise_tmp, tmp_1, tmp_2, x_center, diff, sigma_diff, res, pos_15ADC, acc_15ADC